In [36]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch_geometric as tg
import torch_geometric.utils as tgu

In [54]:
image = torch.from_numpy(np.random.random([2,1,32,32])*128)
category = torch.zeros(2,10)
category[:,1] = 1
timestep = torch.zeros(2,1)
timestep[:,0] = 25

In [70]:
image.shape

torch.Size([2, 1, 32, 32])

In [56]:
cord_x = torch.arange(image.shape[-1]).repeat(image.shape[-2], 1)[None]
cord_y = torch.arange(image.shape[-2]).repeat(image.shape[-1], 1).T[None]
print(cord_x.shape)

torch.Size([1, 32, 32])


In [74]:
def process_image(_image):
    image = _image.reshape([4,16,16])
    cord_x = torch.arange(image.shape[-1]).repeat(image.shape[-2], 1)[None]/64
    cord_y = torch.arange(image.shape[-2]).repeat(image.shape[-1], 1).T[None]/64
    processed = torch.cat([image, cord_x, cord_y])
    return processed.flatten(start_dim=1).T

def build_gnn_batch(**kwargs):
    print({k:v.shape for k,v in kwargs.items()})
    batch_size = list(kwargs.values())[0].shape[0]
    samples = [tg.data.HeteroData() for _ in range(batch_size)]
    # for arg_name, values in kwargs.items():

    for sample, value in zip(samples, kwargs["x"]):
        node_feats = process_image(value)
        print('111 ',node_feats.shape)
        sample["x"].x = node_feats
        adj = torch.ones(node_feats.shape[0], node_feats.shape[0])
        sample["x", 'to', "x"].edge_index = tgu.dense_to_sparse(adj)

    for arg_name in ['t', 'cat']:
        for sample, value in zip(samples, kwargs[arg_name]):
            sample[arg_name].x = value[None]
            adj = torch.ones(1, sample["x"].x.shape[0])
            sample[arg_name, 'to', "x"].edge_index = tgu.dense_to_sparse(adj)
    
    return tg.data.Batch.from_data_list(samples)
    

def extract_prediction(x_batch):
    return x_batch.reshape([-1,16,16,4]).permute([0,3,1,2]).reshape([-1,1,32,32])
    

In [75]:
oo = build_gnn_batch(
    x=image,
    cat=category,
    t=timestep
)

{'x': torch.Size([2, 1, 32, 32]), 'cat': torch.Size([2, 10]), 't': torch.Size([2, 1])}
111  torch.Size([256, 6])
111  torch.Size([256, 6])


In [76]:
extract_prediction(oo['x'].x[:, :4]) == image

tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]]],


        [[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]]]])

In [64]:
oo['x', 'to', 'x'].edge_index[0].shape

torch.Size([2, 131072])

In [65]:
oo['x'].x.dtype

torch.float64

In [66]:
oo.edge_index_dict['x', 'to', 'x'][0]

tensor([[  0,   0,   0,  ..., 511, 511, 511],
        [  0,   1,   2,  ..., 509, 510, 511]])

In [69]:
extract_prediction(oo['x'].x).shape

torch.Size([3, 1, 32, 32])

In [72]:
oo['x'].x.shape

torch.Size([512, 6])